In [24]:
import pandas as pd
import altair as alt

# CSV einlesen (Pfad ggf. anpassen)
df = pd.read_csv("Demografische Daten.csv")

# Altair-Standard: keine Max-Row-Limits
alt.data_transformers.disable_max_rows()

# Spaltennamen (zur Sicherheit einmal sichtbar machen)
df.columns


Index(['Zeitstempel', 'Geben Sie ihr zufälliges Kürzel an',
       'Unter welchem Geschlecht ordnen Sie sich ein?', 'Wie alt sind Sie?',
       'Wie oft spielen Sie Videospiele',
       'Sie sind sehr gut in dem Umgang mit Computern geübt',
       'Welche Geräte verwenden Sie zum Spielen digitaler Spiele?',
       'Wie oft spielen Sie nicht digitale Spiele (Karten oder Brettspiele)?',
       'Erster Durchlauf'],
      dtype='str')

In [25]:
# -----------------------------
# Helper: Prozentwerte berechnen
# -----------------------------
def freq_df(df, col, dropna=True, na_label="Keine Angabe"):
    s = df[col]
    if dropna:
        s = s.dropna()
    else:
        s = s.fillna(na_label)
    out = s.value_counts(dropna=False).rename_axis(col).reset_index(name="count")
    out["percent"] = (out["count"] / len(df) * 100).round(1)
    return out


In [26]:
# -----------------------------
# 1) Geschlecht
# -----------------------------
gender = freq_df(df, "Unter welchem Geschlecht ordnen Sie sich ein?", dropna=True)

gender_chart = alt.Chart(gender).mark_bar().encode(
    x=alt.X("count:Q", title="Anzahl"),
    y=alt.Y("Unter welchem Geschlecht ordnen Sie sich ein?:N", title="Geschlecht", sort="-x"),
    tooltip=[
        alt.Tooltip("Unter welchem Geschlecht ordnen Sie sich ein?:N", title="Geschlecht"),
        alt.Tooltip("count:Q", title="Anzahl"),
        alt.Tooltip("percent:Q", title="Prozent")
    ]
).properties(
    title="Geschlechterverteilung",
    width=450,
    height=200
)

gender_chart


alt.Chart(...)

In [27]:
# -----------------------------
# 2) Alter
# -----------------------------
age = freq_df(df, "Wie alt sind Sie?", dropna=True)

age_chart = alt.Chart(age).mark_bar().encode(
    x=alt.X("count:Q", title="Anzahl"),
    y=alt.Y("Wie alt sind Sie?:N", title="Altersgruppe", sort="-x"),
    tooltip=[
        alt.Tooltip("Wie alt sind Sie?:N", title="Altersgruppe"),
        alt.Tooltip("count:Q", title="Anzahl"),
        alt.Tooltip("percent:Q", title="Prozent")
    ]
).properties(
    title="Altersverteilung",
    width=450,
    height=220
)

age_chart


alt.Chart(...)

In [28]:
# -----------------------------
# 3) Häufigkeit Videospiele
# -----------------------------
play_freq = freq_df(df, "Wie oft spielen Sie Videospiele", dropna=True)

play_freq_chart = alt.Chart(play_freq).mark_bar().encode(
    x=alt.X("count:Q", title="Anzahl"),
    y=alt.Y("Wie oft spielen Sie Videospiele:N", title="Spielhäufigkeit", sort="-x"),
    tooltip=[
        alt.Tooltip("Wie oft spielen Sie Videospiele:N", title="Spielhäufigkeit"),
        alt.Tooltip("count:Q", title="Anzahl"),
        alt.Tooltip("percent:Q", title="Prozent")
    ]
).properties(
    title="Häufigkeit des Videospielens",
    width=450,
    height=220
)

play_freq_chart


alt.Chart(...)

In [29]:
# -----------------------------
# 4) Computerkompetenz (Skala) – Boxplot + Punkte
# -----------------------------
skill_col = "Sie sind sehr gut in dem Umgang mit Computern geübt"
df_skill = df[[skill_col]].dropna()

skill_box = alt.Chart(df_skill).mark_boxplot().encode(
    y=alt.Y(f"{skill_col}:Q", title="Computerkompetenz (Skala)")
).properties(
    title="Selbsteinschätzung Computerkompetenz",
    width=200,
    height=300
)

skill_points = alt.Chart(df_skill).mark_point(size=60, filled=True).encode(
    y=alt.Y(f"{skill_col}:Q"),
    x=alt.value(0),
    tooltip=[alt.Tooltip(f"{skill_col}:Q", title="Wert")]
)

(skill_box + skill_points).properties(width=250)


alt.LayerChart(...)

In [30]:
# -----------------------------
# 5) Nicht-digitale Spiele (Karten/Brettspiele)
# -----------------------------
analog = freq_df(df, "Wie oft spielen Sie nicht digitale Spiele (Karten oder Brettspiele)?", dropna=True)

analog_chart = alt.Chart(analog).mark_bar().encode(
    x=alt.X("count:Q", title="Anzahl"),
    y=alt.Y("Wie oft spielen Sie nicht digitale Spiele (Karten oder Brettspiele)?:N",
            title="Häufigkeit (nicht-digital)", sort="-x"),
    tooltip=[
        alt.Tooltip("Wie oft spielen Sie nicht digitale Spiele (Karten oder Brettspiele)?:N", title="Häufigkeit"),
        alt.Tooltip("count:Q", title="Anzahl"),
        alt.Tooltip("percent:Q", title="Prozent")
    ]
).properties(
    title="Häufigkeit nicht-digitaler Spiele",
    width=450,
    height=220
)

analog_chart


alt.Chart(...)

In [31]:
# -----------------------------
# 6) Geräte (Mehrfachnennungen) – explode
# -----------------------------
devices_col = "Welche Geräte verwenden Sie zum Spielen digitaler Spiele?"
devices = (
    df[devices_col]
    .dropna()
    .astype(str)
    .str.split(r"\s*,\s*")
    .explode()
    .str.strip()
)

devices_df = devices.value_counts().rename_axis("device").reset_index(name="count")
devices_df["percent_of_sample"] = (devices_df["count"] / len(df) * 100).round(1)

devices_chart = alt.Chart(devices_df).mark_bar().encode(
    x=alt.X("count:Q", title="Nennungen"),
    y=alt.Y("device:N", title="Gerät", sort="-x"),
    tooltip=[
        alt.Tooltip("device:N", title="Gerät"),
        alt.Tooltip("count:Q", title="Nennungen"),
        alt.Tooltip("percent_of_sample:Q", title="% der Stichprobe")
    ]
).properties(
    title="Verwendete Geräte (Mehrfachnennungen)",
    width=600,
    height=260
)

devices_chart


alt.Chart(...)

In [32]:
# -----------------------------
# 7) Gruppenzuordnung (Erster Durchlauf)
# -----------------------------
first_run_col = "Erster Durchlauf"
first_run = df[first_run_col].fillna("Keine Kennzeichnung").value_counts().rename_axis("first_run").reset_index(name="count")
first_run["percent"] = (first_run["count"] / len(df) * 100).round(1)

first_run_chart = alt.Chart(first_run).mark_bar().encode(
    x=alt.X("count:Q", title="Anzahl"),
    y=alt.Y("first_run:N", title="Erster Durchlauf", sort="-x"),
    tooltip=[
        alt.Tooltip("first_run:N", title="Erster Durchlauf"),
        alt.Tooltip("count:Q", title="Anzahl"),
        alt.Tooltip("percent:Q", title="Prozent")
    ]
).properties(
    title="Gruppenzuordnung (Erster Durchlauf)",
    width=450,
    height=180
)

first_run_chart


alt.Chart(...)

In [33]:
# -----------------------------
# 8) Alles als Dashboard (2 Zeilen)
# -----------------------------
dashboard = alt.vconcat(
    alt.hconcat(gender_chart, age_chart).resolve_scale(x="independent", y="independent"),
    alt.hconcat(analog_chart, devices_chart).resolve_scale(x="independent", y="independent")
).properties(
    title="Demografische Auswertung (Übersicht)"
)

dashboard


alt.VConcatChart(...)